In [1]:
import json

In [3]:
fname = 'dataset/Pokemon-primary_type/info.json'
with open(fname) as fopen:
        info_dict = json.load(fopen)

In [4]:
col_label = info_dict['label']
eval_metric = info_dict['eval_metric']

In [13]:
from baselines.utils import prepare_ag_dataset

In [14]:
train_df, dev_df, test_df, feature_metadata = prepare_ag_dataset('dataset/Pokemon-primary_type/')

In [16]:
train_df.head()

,name,generation,status,species,type_1,height_m,weight_kg,abilities_number,ability_1,ability_2,...,defense,sp_attack,sp_defense,speed,catch_rate,base_friendship,base_experience,growth_rate,percentage_male,Image Path
0,Shaymin,4,Mythical,Gratitude Pokémon,Grass,0.2,2.1,1,Natural Cure,No,...,100,100,100,100,45,100.0,270.0,Medium Slow,-100.0,dataset/Pokemon-primary_type/train_images/shay...
1,Venusaur,1,Normal,Seed Pokémon,Grass,2.0,100.0,2,Overgrow,No,...,83,100,100,80,45,70.0,236.0,Medium Slow,87.5,dataset/Pokemon-primary_type/train_images/venu...
2,Roserade,4,Normal,Bouquet Pokémon,Grass,0.9,14.5,3,Natural Cure,Poison Point,...,65,125,105,90,75,70.0,232.0,Medium Slow,50.0,dataset/Pokemon-primary_type/train_images/rose...
3,Victreebel,1,Normal,Flycatcher Pokémon,Grass,1.7,15.5,2,Chlorophyll,No,...,65,100,70,70,45,70.0,221.0,Medium Slow,50.0,dataset/Pokemon-primary_type/train_images/vict...
4,Abomasnow,4,Normal,Frost Tree Pokémon,Grass,2.2,135.5,2,Snow Warning,No,...,75,92,85,60,60,70.0,173.0,Slow,50.0,dataset/Pokemon-primary_type/train_images/abom...


In [17]:
from autogluon.multimodal.data.infer_types import infer_column_types

In [18]:
 column_types = infer_column_types(
            data=train_df.drop(columns=col_label),
            valid_data=dev_df.drop(columns=col_label),
        )

In [19]:
print(f'[INFO] column_types={column_types} by infer_column_types()')
text_cols = [_[0] for _ in column_types.items() if _[1] in ['text', 'categorical']]
print(f'[INFO] Text raw input collected from columns: {text_cols}')

[INFO] column_types=OrderedDict([('name', 'categorical'), ('generation', 'categorical'), ('status', 'categorical'), ('species', 'categorical'), ('height_m', 'numerical'), ('weight_kg', 'numerical'), ('abilities_number', 'categorical'), ('ability_1', 'categorical'), ('ability_2', 'categorical'), ('ability_hidden', 'categorical'), ('total_points', 'numerical'), ('hp', 'numerical'), ('attack', 'numerical'), ('defense', 'numerical'), ('sp_attack', 'numerical'), ('sp_defense', 'numerical'), ('speed', 'numerical'), ('catch_rate', 'numerical'), ('base_friendship', 'numerical'), ('base_experience', 'numerical'), ('growth_rate', 'categorical'), ('percentage_male', 'categorical'), ('Image Path', 'categorical')]) by infer_column_types()
[INFO] Text raw input collected from columns: ['name', 'generation', 'status', 'species', 'abilities_number', 'ability_1', 'ability_2', 'ability_hidden', 'growth_rate', 'percentage_male', 'Image Path']


In [27]:
from autogluon.tabular import TabularDataset, FeatureMetadata
from typing import Tuple, Dict, Any
import pandas as pd
import numpy as np

In [28]:
def merge_train_dev_test_dfs(
        train_df: TabularDataset, 
        dev_df: TabularDataset, 
        test_df: TabularDataset,
        col_label: str
        ) -> Tuple[pd.DataFrame, pd.Series, Tuple[np.ndarray, np.ndarray, np.ndarray]]:
    all_data = pd.concat([train_df, dev_df, test_df])
    train_mask = np.zeros((all_data.shape[0]), dtype=bool)
    train_mask[:train_df.shape[0]] = 1
    assert train_mask.sum() == train_df.shape[0]
    dev_mask = np.zeros((all_data.shape[0]), dtype=bool)
    dev_mask[train_df.shape[0]:train_df.shape[0]+dev_df.shape[0]] = 1
    assert dev_mask.sum() == dev_df.shape[0]
    test_mask = np.zeros((all_data.shape[0]), dtype=bool)
    test_mask[train_df.shape[0]+dev_df.shape[0]:] = 1
    assert test_mask.sum() == test_df.shape[0]
    all_data, all_label = all_data.drop(columns=col_label), all_data[col_label]
    return all_data, all_label, (train_mask, dev_mask, test_mask)

In [29]:
all_data, all_label, masks_tuple = merge_train_dev_test_dfs(train_df, dev_df, test_df, col_label)

In [31]:
def pack_cate_text_cols_to_one_sent(elem: Dict[str, Any]) -> str:
    sentence = ''
    for k, v in elem.items():
        sentence = f'{sentence}{k}: {v}. '
    return sentence

In [32]:
text_cols_raw = all_data[text_cols].to_dict('records')
text_cols_raw = [pack_cate_text_cols_to_one_sent(_) for _ in text_cols_raw]

In [34]:
from baselines.utils import IMG_COL

In [35]:
img_paths = all_data[IMG_COL].tolist()

In [37]:
len(img_paths)

897

In [38]:
len(text_cols_raw)

897

In [40]:
from PIL import Image

In [41]:
from transformers import CLIPProcessor, CLIPModel

In [42]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

Downloading:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/577M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/592 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/842k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [44]:
image = Image.open(img_paths[0])

In [46]:
text_cols_raw[0]

'name: Shaymin. generation: 4. status: Mythical. species: Gratitude Pokémon. abilities_number: 1. ability_1: Natural Cure. ability_2: No. ability_hidden: No. growth_rate: Medium Slow. percentage_male: -100.0. Image Path: dataset/Pokemon-primary_type/train_images/shaymin.jpeg. '

In [88]:
inputs = processor(text=text_cols_raw[0], images=image, return_tensors="pt")

In [89]:
inputs['input_ids'] = inputs['input_ids'][:,:75]

In [90]:
inputs['attention_mask'] = inputs['input_ids'][:,:75]

In [91]:
outputs = model(**inputs)

In [93]:
outputs['text_embeds'].shape

torch.Size([1, 512])

In [94]:
outputs['image_embeds'].shape

torch.Size([1, 512])

In [95]:
from sklearn import preprocessing

In [96]:
le = preprocessing.LabelEncoder()
le.fit(train_df[col_label])
all_label = le.transform(all_label)

In [102]:
text_features = []
image_features = []

for i in range(897):
    image = Image.open(img_paths[i])
    text = text_cols_raw[0]
    inputs = processor(text=text, images=image, return_tensors="pt", padding=True)
    inputs['input_ids'] = inputs['input_ids'][:,:75]
    inputs['attention_mask'] = inputs['attention_mask'][:,:75]
    outputs = model(**inputs)
    text_features.append(outputs['text_embeds'])
    image_features.append(outputs['image_embeds'])

In [99]:
import torch

In [103]:
text_features = torch.cat(text_features, dim=0)

In [104]:
image_features = torch.cat(image_features, dim=0)

In [105]:
import pickle

In [106]:
with open('dataset/Pokemon-primary_type/text_features.pkl', 'wb') as fs:
    pickle.dump(text_features, fs)

In [107]:
with open('dataset/Pokemon-primary_type/image_features.pkl', 'wb') as fs:
    pickle.dump(image_features, fs)

In [109]:
# text_feats, image_feats, ti_labels, ti_masks_tuple
# text_features, image_features, all_labels, masks_tuple